# Librairies à installer

In [ ]:
!pip install multidict
!pip install spacy-langdetect
!pip install -U spacy
!python -m spacy download fr_core_news_sm
!python -m spacy download fr_dep_news_trf

     |████████████████████████████████| 160 kB 5.6 MB/s 
     |████████████████████████████████| 998 kB 5.3 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993430 sha256=852de7b2cf7a75b40997cc3c4b7142df26591ff720c7713bc211f9ca99ba6211
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect
     |████████████████████████████████| 6.0 MB 5.2 MB/s 
     |████████████████████████████████| 10.1 MB 19.3 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 451 kB 35.1 MB/s 
     |████████████████████████████████| 628 kB 37.8 MB/s 
     |████████████████████████████████| 181 kB 46.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    U

# Ne pas oublier de relancer l'environnement

In [1]:
from google.colab import files
import os

# Si le dossier est déjà monté dans l'environnement d'exécution, on passe
# Si le dossier est déjà monté dans l'environnement d'exécution, on passe
try:
  os.chdir('DistorsionCom')
except:
  !git clone https://github.com/Patent2net/DistorsionCom.git
  print("Dossier monté, vous pouvez continuer l'exécution")
  os.chdir('DistorsionCom')

stockageEntree = "/content/DistorsionCom"
stockageSortie = "/content/DistorsionCom"

Cloning into  'DistorsionCom'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 228 (delta 75), reused 197 (delta 44), pack-reused 0
Receiving objects: 100% (228/228), 35.80 MiB | 17.69 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Dossier monté, vous pouvez continuer l'exécution



---
Personnalisation

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Récupérer les sorties sur son drive (créer un dossier "OUT")
stockageSortie = "/content/drive/MyDrive/OUT"

---

In [ ]:
from spacy.language import Language
from spacy_langdetect import LanguageDetector

In [ ]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

In [ ]:
# A ne pas run plusieurs fois !!
Language.factory("language_detector", func=get_lang_detector)

<function __main__.get_lang_detector>

In [ ]:
import multidict as multidict

In [3]:
from os import path
import os
import pickle
from collections import Counter
import re 
import spacy
from urllib import parse

In [ ]:
with open (stockageEntree + "/Ressources/NIVEAU1-PNPC.csv", "r") as partners:
  donnee= partners.readlines()

In [ ]:
partnenaires = []
for lig in donnee [1:]:
  urls = lig.split(";") [2]
  partnenaires .extend(urls .split())

# Traitement des backlinks scrappés
> backlinks.pkl est le fichier produit par le scapper. On le charge puis on le spacytraite

## Chargement des données de ref

### Les stats des sites de référence pour être dans le comparable

In [ ]:
fichierEntree = stockageEntree + '/ContenusTraites/' +'stats.pkl'
with open (fichierEntree, 'rb') as  fictemp: #on met tous les contenus dans pickle
  stats = pickle.load(fictemp)


### Chargement des données

In [4]:
fichierEntree = stockageEntree + '/OUT/ContenusBacklinks/' +"backlinks.pkl"
with open (fichierEntree, 'rb') as  fictemp: #pour ouvrir les contenus 
  soup=pickle.load(fictemp)


In [8]:
import json
with open(stockageSortie + '/JSON/BacklinksContenus.json', 'w', encoding='utf-8') as f:
    json.dump(soup, f, ensure_ascii=False, indent=4)

# Filtrage des backlinks

> Exclusion des sites "mal" collectés. Traités par la procédure bs4 un peu violente, il se peut que le contenu de certains sites soit mal récupéré. Exclusion des url sur lesquels moins de 200 caractères ont été récupérés.

In [ ]:
urlAGarder = [url for url in soup .keys() if len (soup [url]) > 200] # 200 est arbitraire
urlVirés = [url for url in soup .keys() if len (soup [url]) <= 200] 

In [ ]:
import pprint as pp
pp.pprint(stats)

In [ ]:
tempVerb = dict() # fonction du ndf
tempNoun = dict()
tempAdj = dict()
tempLoc = dict()
tempMisc = dict()
tempPer = dict()
tempOrg = dict()
dicoEtiquettage = dict()
aEviter = dict()
verbes = dict() #listes à remplir avec verbes/adj/noms/entités trouvés 
adjectifs  = dict() 
noms = dict()
adv = dict()

In [ ]:
tal = spacy.load('fr_core_news_sm')
# 'fr_dep_news_trf' fonctionne mieux pour le reste
tal1 = spacy.load('fr_dep_news_trf', disable = ['NER'])



## Détection de langue
> les procédures de lemmatisation et autres fonctions TAL sont fonction de la langue. Seul le modèle FR est chargé. Les autres langues seront exclues.

In [ ]:
tal1.add_pipe('language_detector', last=True) # Permet de détecter la langue

In [ ]:
tal.add_pipe('language_detector', last=True) # Permet de détecter la langue



---


## <a id="deconnexion"></a> Gestion des déconnexions colab

> Entre autres : si le code plante sur un url on sauvegarde à chaque url réalisé. Le code qui suit permet de récupérer le contenu sauvegardé pour éviter d'avoir à nouveau à traiter

*Note : si le code du **spacifycateur** est changé il faut supprimer le contenu du dossier de sortie, ici "/content/drive/MyDrive/Com-Web/Projet Master2 Langues/OUT/ContenusTraites2/"*

In [ ]:
# déjà vus

if "spacySites.pkl" in os.listdir(stockageSortie + '/OUT/backlinksTraites/'):
  fichierDeSortie = stockageSortie + '/OUT/backlinksTraites/' +'spacySites.pkl'
  with open (fichierDeSortie, 'rb') as  fictemp: #on met tous les contenus dans pickle
    dicoEtiquettage = pickle.load(fictemp)
  fichierDeSortie = stockageSortie + '/OUT/backlinksTraites/' +'verbesSites.pkl'
  with open (fichierDeSortie, 'rb') as  fictemp: #on met tous les contenus dans pickle
    verbes =  pickle.load(fictemp)

  fichierDeSortie = stockageSortie + '/OUT/backlinksTraites/' +'nomsSites.pkl'
  with open (fichierDeSortie, 'rb') as  fictemp: #on met tous les contenus dans pickle
      noms= pickle.load(fictemp)

  fichierDeSortie = stockageSortie + '/OUT/backlinksTraites/' +'adjectifsSites.pkl'
  with open (fichierDeSortie, 'rb') as  fictemp: #on met tous les contenus dans pickle
      adjectifs = pickle.load(fictemp)
  fichierDeSortie = stockageSortie + '/OUT/backlinksTraites/' +'adverbesSites.pkl'
  with open (fichierDeSortie, 'rb') as  fictemp: #on met tous les contenus dans pickle
      adv = pickle.load(fictemp)
if "SitesAVirer.pkl" in os.listdir(stockageSortie + '/OUT/backlinksTraites/'):  
  fichierDeSortie = stockageSortie + '/OUT/backlinksTraites/' +'SitesAVirer.pkl'
  with open (fichierDeSortie, 'rb') as  fictemp: #on met tous les contenus dans pickle
   urlVirés = pickle.load(fictemp)


In [ ]:
len(urlVirés)

320

## Exclusion des déjàs vus

In [ ]:
print ("avant ", len (urlAGarder))
urlAGarder = [url for url in soup.keys() if url not in dicoEtiquettage .keys()]
print ("après exclusion des traités ",len (urlAGarder))


avant  728
après exclusion des traités  864


In [ ]:
urlAGarder = [url for url in urlAGarder if url not in urlVirés]
print ("après exclusion des virés ",len (urlAGarder))

après exclusion des virés  728




---


# Spacyficateur
> Après avoir détecté les entités nommées, le code suivant pour chaque URL :
- détecte si la langue du contenu de l'url en cours est fr (sinon exclue l'url)
- détecte les entités nommées
- Sur l'ensemble des mots de l'URL :
  - si le mot n'est pas une entité nommée le lemme du mot est "rangé" dans :
    - verbes [url] si c'est un verbe 
    - adjectifs [url] si c'est un adjectif
    - noms[url] si c'est un nom
dicoEtiquettage [url]["ADJ"] = adjectifs [url]
etc.

Note: les variables verbes, adjectifs et noms sont redondantes avec spacySites et pas utiles ici car seront recalculées

In [ ]:
for url in soup .keys(): 
    if url not in urlVirés and url in urlAGarder:
      # nettoyage
      print (url)
      tousLesContenus = soup [url]  
      # deux modèles de données spacy sont utilisés du fait de leur efficacité relative à la détection des entités
      # 'fr_core_news_sm' fonctionne mieux pour la détection des entités
      try:
        doc = tal(str(tousLesContenus))
      except: # exclusion des sites qui passent mal
        print ("viré : ", url)
        urlVirés .append(url)
        fichierDeSortie =  stockageSortie + '/OUT/backlinksTraites/' +'SitesAVirer.pkl'
        with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
          pickle.dump(urlVirés, fictemp)
        continue
      if not doc._.language["language"] == "fr": # exclusion si fr non détecté
        print ("pas fr : ", url)
        urlVirés .append(url)
        fichierDeSortie =  stockageSortie + '/OUT/backlinksTraites/' +'SitesAVirer.pkl'
        with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
          pickle.dump(urlVirés, fictemp)
        continue
      aEviter [url] = []
      entites = []
      if url not in dicoEtiquettage.keys():
        dicoEtiquettage [url] = dict()
      for entity in doc.ents:
        if entity.label_ not in dicoEtiquettage [url] .keys():
          dicoEtiquettage [url] [entity.label_] = []
          dicoEtiquettage [url] [entity.label_] .append(entity.text)
          entites.append(entity.label_)
        else:
          dicoEtiquettage [url] [entity.label_] .append(entity.text)
        #entites.append(entity.text)
        #dicoEtiquettage [url] [entity.label_] = entites #dico: [étiquettage de entité donné par spacy] = la liste des entités
      aEviter [url] = []
      if 'LOC' in dicoEtiquettage [url] .keys():
          aEviter [url] = dicoEtiquettage [url] ['LOC']
      if 'MISC' in dicoEtiquettage [url] .keys():
          aEviter [url] += dicoEtiquettage [url] ['MISC']
      if 'PER' in dicoEtiquettage [url] .keys():
          aEviter [url] += dicoEtiquettage [url] ['PER']
      if 'ORG' in dicoEtiquettage [url] .keys():
          aEviter [url] += dicoEtiquettage [url] ['ORG']


http://www.vanoise-parcnational.fr/fr
http://www.pyrenees-parcnational.fr/fr
http://www.cevennes-parcnational.fr/fr
http://www.mercantour-parcnational.fr/fr
http://www.guadeloupe-parcnational.fr/fr
http://www.parc-amazonien-guyane.fr/fr
http://www.reunion-parcnational.fr/fr
http://www.calanques-parcnational.fr/fr
http://www.forets-parcnational.fr/fr
http://www.parcsnationaux.fr/fr
http://www.portcrosparcnational.fr/
https://www.ramatuelle-tourisme.com/
https://www.lacroixvalmer.fr/
http://www.var.gouv.fr/acces-aux-massifs-forestiers-du-var-a2898.html
http://www.parks.it/world/FR/Eindex.html
pas fr :  http://www.parks.it/world/FR/Eindex.html
https://www.lecampingsauvage.fr/legislation-et-reglementation/camping-sauvage-bivouac
https://en.wikipedia.org/wiki/List_of_northernmost_items
pas fr :  https://en.wikipedia.org/wiki/List_of_northernmost_items
https://www6.sophia.inrae.fr/jardin_thuret
https://fr.wikipedia.org/wiki/Île_de_Porquerolles
https://www.fol83laligue.org/
https://lavandou-p

In [ ]:
keep = [url for url in dicoEtiquettage .keys() if url not in urlVirés and url in aEviter.keys()]

Si le code ci dessous plante (le notebook affiche rouge), il convient de réexécuter les cellules depuis [la gestion des deconnexions collab](#deconnexion)

In [ ]:
# TAL
# Pour nos jeux de données ces traitements plantent et demandent de rééxécuter 
for url in urlAGarder: 
    if url not in urlVirés:  
      ndf = url
      #dicoEtiquettage [ndf] = dict()
      tousLesContenus = soup [url]  
      tempo = max([len(mot) for mot in tousLesContenus .split()])
      print(tempo, url)
      if tempo>100:
        urlVirés .append(url)
        continue # on ignore l'url

        # utilisation de l'autre modèle
      try: 
        doc = tal1(str(tousLesContenus))
      except:
        print ("Spacy Explosion", url)
        urlVirés .append(url)
        fichierDeSortie =  stockageSortie + '/OUT/backlinksTraites/' +'SitesAVirer.pkl'
        with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
          pickle.dump(urlVirés, fictemp)
        continue
      if ndf not in verbes.keys():
          verbes [ndf] = []
      if ndf not in adjectifs.keys():
          adjectifs [ndf] = []
      if ndf not in noms.keys():
          noms [ndf] = []
      if ndf not in adv.keys():
          adv [ndf] = []
  
      for truc in doc: 
          if truc.text not in aEviter [ndf] and truc.text.lower() not in aEviter [ndf]: 
            if len(truc.text)>2: #évite directement les articles et erreurs à la noix
              if truc.pos_ == "VERB":
                verbes [ndf].append(truc.lemma_) #ajoute à la liste (.lemma_ fait la lemmatisation)
                dicoEtiquettage [ndf][truc.pos_] = verbes [ndf] #ajoute dans le dico: ["VERB"] = la liste des verbes
              if truc.pos_ == "ADJ":
                adjectifs [ndf].append(truc.lemma_) 
                dicoEtiquettage [ndf][truc.pos_] = adjectifs [ndf] #dico:["ADJ"] = la liste des adjectifs
              if truc.pos_ == "NOUN":
                noms [ndf].append(truc.lemma_)
                dicoEtiquettage [ndf][truc.pos_] = noms [ndf] #dico: ["NOUN"] = la liste des noms
              if truc.pos_ == "ADV":
                adv [ndf].append(truc.lemma_)
                dicoEtiquettage [ndf][truc.pos_] = adv [ndf] #dico: ["NOUN"] = la liste des adverbes

      fichierDeSortie =  stockageSortie + '/OUT/backlinksTraites/' +'spacySites.pkl'
      with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
          pickle.dump(dicoEtiquettage, fictemp)
      fichierDeSortie =  stockageSortie + '/OUT/backlinksTraites/' +"verbesSites.pkl"
      with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
        pickle.dump(verbes, fictemp)
      fichierDeSortie =  stockageSortie + '/OUT/backlinksTraites/' +"adverbesSites.pkl"
      with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
        pickle.dump(adv, fictemp)
      fichierDeSortie =  stockageSortie + '/OUT/backlinksTraites/' +'nomsSites.pkl'
      with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
        pickle.dump(noms, fictemp)

      fichierDeSortie =  stockageSortie + '/OUT/backlinksTraites/' +'adjectifsSites.pkl'
      with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
        pickle.dump(adjectifs, fictemp)


19 http://www.vanoise-parcnational.fr/fr
14 http://www.pyrenees-parcnational.fr/fr
19 http://www.cevennes-parcnational.fr/fr
41 http://www.mercantour-parcnational.fr/fr
17 http://www.guadeloupe-parcnational.fr/fr
15 http://www.parc-amazonien-guyane.fr/fr
20 http://www.reunion-parcnational.fr/fr
15 http://www.calanques-parcnational.fr/fr
15 http://www.forets-parcnational.fr/fr
16 http://www.parcsnationaux.fr/fr
19 http://www.portcrosparcnational.fr/
25 https://www.ramatuelle-tourisme.com/
12 https://www.lacroixvalmer.fr/
25 http://www.var.gouv.fr/acces-aux-massifs-forestiers-du-var-a2898.html
24 https://www.lecampingsauvage.fr/legislation-et-reglementation/camping-sauvage-bivouac
59 https://www6.sophia.inrae.fr/jardin_thuret
16 https://fr.wikipedia.org/wiki/Île_de_Porquerolles
85 https://www.fol83laligue.org/
16 https://lavandou-plongee.com/
21 https://divingiens.fr/
14 http://www.ulysseplongee.fr/


Token indices sequence length is longer than the specified maximum sequence length for this model (605 > 512). Running this sequence through the model will result in indexing errors


87 https://www.portland-bateau.com/
19 https://www.azur-hotel.org/
19 https://www.rrenpaca.org/
20 https://www.ciplavandou.com/
19 http://alinea.hautetfort.com/
22 http://www.bandol-blog.com/
17 https://campingshyeres.com/
15 https://www.portshyeres.fr/fr/
139 https://www.randonner-malin.com/bivouacs-et-camping-sauvage-en-france/
34 https://ofb.gouv.fr/provence-alpes-cote-dazur
16 http://www.port-cros.net/
71 http://www.pommiers.com/olive/olivier.htm
55 https://www.nouvelobs.com/voyage/20200424.OBS27940/10-spots-en-france-qui-ont-un-gout-de-bout-du-monde.html
18 http://fort-eminence-port-cros.blogspot.com/
25 http://www.cpshyeres.com/
22 http://blog.worstindustries.fr/
15 http://www.restaurant-trinquette-port-cros.fr/
16 https://www.gite-de-plongee-du-gapeau.com/
69 http://www.conservatoire-du-littoral.fr/siteLittoral/21/28-cap-lardier-83_var.htm
27 http://vexil.prov.free.fr/toulon%20provence%20m%E9diterran%E9e/tpme.html
15 http://www.acroporis.org/index.php
95 http://www.declic-bleu-m

# Sauvegarde des traitements

> Normalement ce qui suit est exécuté au fil de la boucle sur les URL.

In [ ]:
fichierDeSortie = stockageSortie + '/backlinksTraites/' +'spacySites.pkl'
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(dicoEtiquettage, fictemp)


In [ ]:
fichierDeSortie = stockageSortie + '/backlinksTraites/' +'stats.pkl'
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(stats, fictemp)


In [ ]:
fichierDeSortie = stockageSortie + '/backlinksTraites/' +"verbesSites.pkl"
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(verbes, fictemp)

fichierDeSortie = stockageSortie + '/backlinksTraites/' +'nomsSites.pkl'
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(noms, fictemp)

fichierDeSortie = stockageSortie + '/backlinksTraites/' +'adjectifsSites.pkl'
with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans pickle
  pickle.dump(adjectifs, fictemp)

In [ ]:
dicoEtiquettage .values()